## PGA Draftkings Notebook
Use [PGA Website](https://www.pgatour.com/tournaments/schedule.html) to look up tournament info and fill out first USER INPUT block below. (Fetch/XHR from Network tab, Paylod sub-tab)

Looks like the tournament ID is also in the address bar.

### User Input

In [1]:
# === USER INPUTS ===
# Old Tournament
old_tournament_name = "3M Open"
tournament_date = "7/27/2025"  # Ending date of tournament
old_course = "TPC Twin Cities"
tournament_id = "R2025525"  # Tournament ID from the PGA Tour API - Also visible in the URL of the tournament page

# New Tournament
new_tournament_name = "Wyndham Championship"
new_ending_date = "8/3/2025"
new_course = "Sedgefield Country Club"
new_season = 2025


# === LIBRARIES AND VARIABLES ===
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
import sqlite3 as sql
import numpy as np
from numpy import nan
import os
import importlib
import utils.db_utils
from utils.db_utils import TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP

# === TOURNAMENT CONFIG ===
tournament_config = {
    "old": {
        "name": old_tournament_name,
        "date": tournament_date,
        "course": old_course,
        "id": tournament_id
    },
    "new": {
        "name": new_tournament_name,
        "course": new_course,
        "season": new_season,
        "ending_date": pd.to_datetime(new_ending_date, format="%m/%d/%Y"),
        "quoted_course": f'"{new_course}"',
        "quoted_name": f"'{new_tournament_name}'"
    }
}

this_week_key = str(tournament_config["new"]["ending_date"].date())

# Create a minimal one-row history DataFrame
this_week_history = pd.DataFrame([{
    "SEASON": tournament_config["new"]["season"],
    "TOURNAMENT": tournament_config["new"]["name"],
    "ENDING_DATE": tournament_config["new"]["ending_date"],
    "COURSE": tournament_config["new"]["course"],
    "TOURN_ID": None
}])

### Update Database

#### Old Tournament

In [2]:
importlib.reload(utils.db_utils)  # Only needed if you're actively editing db_utils.py
from utils.db_utils import update_tournament_results

# Change these each year!!
season = 2025
year = 20250  # Unique GraphQL year distinguishing number in case of multiple per year

# Run the update
db_path = "data/golf.db"  # Or use os.path.join("data", "golf.db")
tournDf = update_tournament_results(tournament_config, db_path, season, year)

# Show just the most recent tournament added for confirmation
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{db_path}")

query = f"""
SELECT *
FROM tournaments
WHERE TOURN_ID = '{tournament_config['old']['id']}'
  AND ENDING_DATE = '{datetime.strptime(tournament_config['old']['date'], '%m/%d/%Y').date()}'
"""

recent = pd.read_sql(query, engine)
engine.dispose()
recent.sort_values(by='FINAL_POS').head()

📦 Fetching results for tournament ID R2025525 (3M Open), year: 20250
✅ 156 new rows added for '3M Open'


,SEASON,ENDING_DATE,TOURN_ID,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,ROUNDS:1,ROUNDS:2,ROUNDS:3,ROUNDS:4,OFFICIAL_MONEY,FEDEX_CUP_POINTS
0,2025,2025-07-27,R2025525,3M Open,TPC Twin Cities,Kurt Kitayama,1,1,-6,E,-11,-6,"$1,512,000.00",500.000
1,2025,2025-07-27,R2025525,3M Open,TPC Twin Cities,Sam Stevens,2,2,-9,-3,-5,-5,"$915,600.00",300.000
2,2025,2025-07-27,R2025525,3M Open,TPC Twin Cities,Matt Wallace,T3,3,-6,-4,-3,-7,"$410,025.00",133.750
3,2025,2025-07-27,R2025525,3M Open,TPC Twin Cities,David Lipsky,T3,3,-3,-3,-7,-7,"$410,025.00",133.750
4,2025,2025-07-27,R2025525,3M Open,TPC Twin Cities,Pierceson Coody,T3,3,-6,-6,-4,-4,"$410,025.00",133.750


#### Stats

In [3]:
importlib.reload(utils.db_utils)
from utils.db_utils import update_season_stats  # <- This line is essential

# Change these each year!!
statsYear = 2025

stats_df = update_season_stats(statsYear, db_path)
stats_df.head()

✅ Overwrote stats for season 2025 with 998 rows.


,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,...,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON
0,A.J. Ewart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,618,.1979,2025
1,Aaron Baddeley,173.0,-1.327,175.0,-1.232,169.0,-0.543,9.0,0.447,158.0,...,289.8,120.0,57.27%,175.0,59.76%,85.0,59.60%,514,.2509,2025
2,Aaron Cockerill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,347,.4169,2025
3,Aaron Rai,29.0,0.724,22.0,0.390,57.0,0.246,64.0,0.087,108.0,...,288.6,2.0,73.37%,23.0,69.53%,93.0,59.26%,35,2.5936,2025
4,Aaron Wilkin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,891,.1041,2025


#### Odds
Not usually needed for weekly routine.

**Manual Fix! Odds name cleanup (only needed when joins fail)**

Make sure to update the dictionaries in db_utils.py if new names need to be added.

In [19]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_odds_names, PLAYER_NAME_MAP, TOURNAMENT_NAME_MAP

db_path = "data/golf.db" 
updated_odds = clean_odds_names(db_path, TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP)
updated_odds.head()

✅ Cleaned and updated 207 rows in 'odds' table.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,TOURNAMENT_ORIG,PLAYER_ORIG
5797,2024,The Open Championship,2024-07-21,Rory McIlroy,17/2,8.5,Open Championship,Rory McIlroy
5798,2024,The Open Championship,2024-07-21,Jon Rahm,20/1,20.0,Open Championship,Jon Rahm
5799,2024,The Open Championship,2024-07-21,Scottie Scheffler,11/2,5.5,Open Championship,Scottie Scheffler
5800,2024,The Open Championship,2024-07-21,Viktor Hovland,30/1,30.0,Open Championship,Viktor Hovland
5801,2024,The Open Championship,2024-07-21,Cameron Smith,50/1,50.0,Open Championship,Cameron Smith


**Historical Odds Updates**

Only run this when loading in entire year odds at the start of each year or if corrections need to be made.  This will load in the entire year into the database and update it using the dictionary in db_utils.py.

In [15]:
importlib.reload(utils.db_utils)
from utils.db_utils import import_historical_odds

oddsYear = "2022-2023"    # URL segment
season = 2023             # PGA Tour season
db_path = "data/golf.db"

odds_df = import_historical_odds(oddsYear, season, db_path)
odds_df.head()

✅ Inserting 120 new rows into odds table...


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
4,2023,Fortinet Championship,2022-09-18,Hideki Matsuyama,18/1,18.0
5,2023,Fortinet Championship,2022-09-18,Max Homa,18/1,18.0
6,2023,Fortinet Championship,2022-09-18,Corey Conners,20/1,20.0
7,2023,Fortinet Championship,2022-09-18,Maverick McNealy,25/1,25.0
8,2023,Fortinet Championship,2022-09-18,Taylor Pendrith,25/1,25.0


**Not normally needed**

The code below is a way to troubleshoot the odds function that is in db_utils.py.  If it does not pull the stats correctly, we can run it outside of the db_utils.py file and explore what might be wrong with it, iterating until it works, and then using that fix to update db_utils.py.  This is not normally needed, but wanted to leave this in place as a reference because this odds website can be tricky.

In [16]:
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime
from io import StringIO

# === USER INPUT ===
oddsYear = "2020-2021"    # URL segment
season = 2021        # PGA Tour season

url = f"http://golfodds.com/archives-{oddsYear}.html"
response = requests.get(url)
tables = pd.read_html(StringIO(response.text))
# raw_df = tables[5]  # the actual table of interest
# Find the largest 2-column table that contains at least some odds-like strings
raw_df = None
for tbl in tables:
    if tbl.shape[1] == 2 and tbl.shape[0] > 50:  # Rough filter
        sample = tbl.iloc[:, 1].astype(str).str.contains(r"\d+/\d+").sum()
        if sample > 5:
            raw_df = tbl
            break

if raw_df is None:
    raise ValueError("❌ Could not find valid odds table on the page.")

# === STEP 1: Initial clean-up ===
df = raw_df.dropna(how="all").reset_index(drop=True)
df.columns = ["PLAYER", "ODDS"]

# 🔧 Clean up non-breaking spaces and extra whitespace
df["PLAYER"] = (
    df["PLAYER"]
    .astype(str)
    .str.replace("\xa0", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df.insert(loc=0, column="SEASON", value=season)
df.insert(loc=1, column="TOURNAMENT", value=np.nan)
df.insert(loc=2, column="ENDING_DATE", value=np.nan)

# === STEP 2: Helper function for parsing date strings ===
def parse_ending_date(text):
    import re
    from datetime import datetime

    # Normalize whitespace and symbols
    text = (
        text.replace("\u2013", "-")
            .replace("–", "-")
            .replace("\xa0", " ")
    )
    text = re.sub(r"\bSept(?!ember)\b", "Sep", text)

    # ✅ Fix typo: "Match" → "March" only when used in a date context
    text = re.sub(r"\bMatch(?=\s+\d{1,2}\s*[-–]\s*\d{1,2},\s*\d{4})", "March", text)

    # Pattern 1: "July 30 - August 2, 2015" or "Oct 29 - Nov 1, 2015"
    match = re.search(r"(\w+)\s\d+\s*-\s*(\w+)\s(\d+),\s(\d{4})", text)
    if match:
        month2, day2, year = match.group(2), match.group(3), match.group(4)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month2} {day2}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 2: "November 21-24, 2024"
    match = re.search(r"(\w+)\s\d+-\d+,\s(\d{4})", text)
    if match:
        month, year = match.group(1), match.group(2)
        day = re.search(r"(\d+)-(\d+)", text).group(2)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month} {day}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 3: "Sunday, October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%A, %B %d, %Y").date()
    except ValueError:
        pass

    # Pattern 4: "October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%B %d, %Y").date()
    except ValueError:
        pass

    return None

last_tourn_name = None
last_date = None

# === STEP 3: Iterate block by block ===
final_rows = []
i = 0
last_tourn_name = None
last_end_date = None

while i < len(df) - 4:
    player_i = str(df.loc[i, "PLAYER"])
    player_i2 = str(df.loc[i + 2, "PLAYER"])
    player_i3 = str(df.loc[i + 3, "PLAYER"]).lower()

    # Detect start of a new tournament block
    is_header = (
        pd.isna(df.loc[i, "ODDS"]) and
        pd.isna(df.loc[i + 1, "ODDS"]) and (
            re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", player_i2) or
            re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", player_i2)
        )
    )

    if is_header:
        tourn_name = player_i.strip()
        end_date = parse_ending_date(player_i2)

        # Skip cancelled or empty blocks
        if "cancelled" in player_i3:
            print(f"⚠️ Skipping cancelled tournament: {tourn_name} — {end_date}")
            i += 4
            continue

        # Avoid duplicate block processing
        if tourn_name == last_tourn_name and end_date == last_end_date:
            i += 1
            continue

        print(f"📍 Detected: {tourn_name} — Ending: {end_date}")
        last_tourn_name = tourn_name
        last_end_date = end_date
        i += 4  # Skip header lines

        # Collect all player rows until next header block
        while i < len(df) - 2:
            next_i2 = str(df.loc[i + 2, "PLAYER"])
            is_next_header = (
                pd.isna(df.loc[i, "ODDS"]) and
                pd.isna(df.loc[i + 1, "ODDS"]) and (
                    re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", next_i2) or
                    re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", next_i2)
                )
            )
            if is_next_header:
                break

            if pd.notna(df.loc[i, "ODDS"]):
                row = df.loc[i].copy()
                row["TOURNAMENT"] = tourn_name
                row["ENDING_DATE"] = end_date
                final_rows.append(row)
            i += 1
    else:
        i += 1

# === STEP 4: Create cleaned DataFrame ===
clean_df = pd.DataFrame(final_rows)

# ✅ Prevent crash if nothing was parsed
if clean_df.empty or "PLAYER" not in clean_df.columns:
    print(f"⚠️ No valid tournament blocks detected for season {season} ({oddsYear})")
    final_df = pd.DataFrame()  # Safe fallback
else:
    # Remove winner tag
    clean_df["PLAYER"] = clean_df["PLAYER"].str.replace(r"\s\*Winner\*", "", regex=True)

    # Clean odds to numeric
    clean_df["VEGAS_ODDS"] = (
        clean_df["ODDS"]
        .str.replace(",", "")
        .str.extract(r"(\d+)/(\d+)")
        .astype(float)
        .apply(lambda x: x[0] / x[1], axis=1)
    )

    # Final output with source index for debugging
    final_df = clean_df[
        ["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER", "ODDS", "VEGAS_ODDS"]
    ].reset_index(drop=True)

    # Drop non-standard team events (e.g., Presidents Cup, Ryder Cup)
    drop_terms = ["Presidents Cup", "Ryder Cup"]
    final_df = final_df[~final_df["TOURNAMENT"].str.contains("|".join(drop_terms), case=False, na=False)]

    display(final_df.head())



📍 Detected: Safeway Open — Ending: 2020-09-13
📍 Detected: US Open — Ending: 2020-09-20
📍 Detected: R & C Championship — Ending: 2020-09-27
📍 Detected: at Big Cedar Lodge - — Ending: 2020-09-22
📍 Detected: Sanderson Farms Champ — Ending: 2020-10-04
📍 Detected: Shriners H for C Open — Ending: 2020-10-11
📍 Detected: The CJ Cup — Ending: 2020-10-18
📍 Detected: ZOZO CHAMPIONSHIP — Ending: 2020-10-25
📍 Detected: Bermuda Championship — Ending: 2020-11-01
📍 Detected: Vivint Houston Open — Ending: 2020-11-08
📍 Detected: The Masters — Ending: 2020-11-15
📍 Detected: The RSM Classic — Ending: 2020-11-22
📍 Detected: Champions for Change — Ending: 2020-11-27
📍 Detected: Mayakoba Golf Classic — Ending: 2020-12-06
📍 Detected: QBE Shootout — Ending: 2020-12-13
📍 Detected: Sentry Tourn of Champions — Ending: 2021-01-10
📍 Detected: Sony Open in Hawaii — Ending: 2021-01-17
📍 Detected: The American Express — Ending: 2021-01-24
📍 Detected: Abu Dhabi HSBC Champ — Ending: 2021-01-24
📍 Detected: Farmers Insura

,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
0,2021,Safeway Open,2020-09-13,Phil Mickelson,20/1,20.0
1,2021,Safeway Open,2020-09-13,Si Woo Kim,20/1,20.0
2,2021,Safeway Open,2020-09-13,Brendan Steele,20/1,20.0
3,2021,Safeway Open,2020-09-13,Shane Lowry,25/1,25.0
4,2021,Safeway Open,2020-09-13,Sergio Garcia,30/1,30.0


In [17]:
from datetime import datetime, date
# ✅ Check for non-date types in ENDING_DATE
non_dates = final_df[~final_df["ENDING_DATE"].apply(lambda x: isinstance(x, date))]

print(f"🧪 Rows with invalid ENDING_DATE values: {len(non_dates)}")
display(non_dates.head(10))



🧪 Rows with invalid ENDING_DATE values: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


In [18]:
dupes = final_df.duplicated(subset=["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER"], keep=False)

print(f"🚨 Duplicate primary keys in final_df: {dupes.sum()}")
display(final_df[dupes].sort_values(by=["SEASON", "TOURNAMENT", "PLAYER"]))

🚨 Duplicate primary keys in final_df: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


#### Misc. Cleaning
Not normally needed

In [26]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_player_names_in_table, PLAYER_NAME_MAP

db_path = "data/golf.db"

# Run for all relevant tables
clean_player_names_in_table(db_path, "tournaments", PLAYER_NAME_MAP)
clean_player_names_in_table(db_path, "stats", PLAYER_NAME_MAP)
clean_player_names_in_table(db_path, "odds", PLAYER_NAME_MAP)

ℹ️ No player names needed updates in 'tournaments'.
ℹ️ No player names needed updates in 'stats'.
ℹ️ No player names needed updates in 'odds'.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,PLAYER_ORIG


## Historical Data

### Pull Relevant Seasons
Do a check to see when this course or tournament have been historically played.

In [4]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_combined_history_seasons

# === USER INPUT ===
seasons = list(range(2016, 2025))  # Adjust as needed
db_path = "data/golf.db"

# Pull course and tournament from config
n_course = tournament_config["new"]["course"]
n_tourn = tournament_config["new"]["name"]

# Fetch relevant history
history_df = get_combined_history_seasons(db_path, course=n_course, tournament=n_tourn, allowed_seasons=seasons)
history_df.head(20)


ℹ️ Found 9 relevant tournaments from course or tournament name.


,SEASON,COURSE,TOURN_ID,TOURNAMENT,ENDING_DATE
779,2016,Sedgefield Country Club,013,Wyndham Championship,2016-08-21
623,2017,Sedgefield Country Club,013,Wyndham Championship,2017-08-20
467,2018,Sedgefield Country Club,013,Wyndham Championship,2018-08-19
312,2019,Sedgefield Country Club,013,Wyndham Championship,2019-08-04
156,2020,Sedgefield Country Club,013,Wyndham Championship,2020-08-16
0,2021,Sedgefield Country Club,013,Wyndham Championship,2021-08-15
1088,2022,Sedgefield Country Club,013,Wyndham Championship,2022-08-07
1244,2023,Sedgefield Country Club,R2023013,Wyndham Championship,2023-08-06
1400,2024,Sedgefield Country Club,R2024013,Wyndham Championship,2024-08-11


### Cut Percentage and FedEx Points
Use a rolling-window approach to look at the most recent cut percentage and how many FedEx cup points have been accumulated recently. This will intentionally not match the PGA Tour stats that start over every year, but will have the same amount of data all the time.  We also add a new feature called Form Density which divides the FedEx Cup Points by the Total Events.

In [5]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts = get_cut_and_fedex_history("data/golf.db", history_df, window_months=9)
# cuts["2024-05-12"].head(20)

for end_date, df in cuts.items():
    print(f"\n📆 {end_date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-08-21 — Wyndham Championship (524 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,16,9,334.0,56.2,20.88,2,2016-08-21,Wyndham Championship
1,Aaron Wise,2,1,0.0,50.0,0.00,1,2016-08-21,Wyndham Championship
2,Abraham Ancer,9,4,96.5,44.4,10.72,2,2016-08-21,Wyndham Championship



📆 2017-08-20 — Wyndham Championship (519 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,14,7,254.14,50.0,18.15,1,2017-08-20,Wyndham Championship
1,Aaron Rai,1,0,0.00,0.0,0.00,0,2017-08-20,Wyndham Championship
2,Aaron Wise,2,2,0.00,100.0,0.00,2,2017-08-20,Wyndham Championship



📆 2018-08-19 — Wyndham Championship (552 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.00,0.0,0.00,0,2022-08-07,Wyndham Championship
1,Aaron Baddeley,6,3,53.94,50.0,8.99,2,2022-08-07,Wyndham Championship
2,Aaron Beverly,1,0,0.00,0.0,0.00,0,2022-08-07,Wyndham Championship



📆 2023-08-06 — Wyndham Championship (604 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,17,11,339.536,64.7,19.97,1,2023-08-06,Wyndham Championship
1,Aaron Beverly,1,1,0.000,100.0,0.00,1,2023-08-06,Wyndham Championship
2,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2023-08-06,Wyndham Championship



📆 2024-08-11 — Wyndham Championship (585 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,14,10,155.725,71.4,11.12,2,2024-08-11,Wyndham Championship
1,Aaron Cockerill,2,0,0.000,0.0,0.00,0,2024-08-11,Wyndham Championship
2,Aaron Rai,18,15,851.657,83.3,47.31,11,2024-08-11,Wyndham Championship


### Recent Form

In [6]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish("data/golf.db", history_df, window_months=9)

# Example preview
for date, df in recent_form.items(): 
    print(f"\n📆 {date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-08-21 — Wyndham Championship (524 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Wesley Bryan,1,8.0,11.54,2016-08-21,Wyndham Championship
1,Jared du Toit,1,9.0,12.98,2016-08-21,Wyndham Championship
2,Tyrrell Hatton,3,10.7,7.72,2016-08-21,Wyndham Championship



📆 2017-08-20 — Wyndham Championship (519 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Matthew Southgate,1,6.0,8.66,2017-08-20,Wyndham Championship
1,Jordan L Smith,1,9.0,12.98,2017-08-20,Wyndham Championship
2,Keith Mitchell,1,11.0,15.87,2017-08-20,Wyndham Championship



📆 2018-08-19 — Wyndham Championship (552 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Anthony Quayle,1,15.0,21.64,2022-08-07,Wyndham Championship
1,Martin Contini,1,16.0,23.08,2022-08-07,Wyndham Championship
2,Rory McIlroy,13,16.0,6.06,2022-08-07,Wyndham Championship



📆 2023-08-06 — Wyndham Championship (604 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Scottie Scheffler,19,6.2,2.07,2023-08-06,Wyndham Championship
1,Grant Forrest,1,11.0,15.87,2023-08-06,Wyndham Championship
2,Henrik Stenson,1,13.0,18.76,2023-08-06,Wyndham Championship



📆 2024-08-11 — Wyndham Championship (585 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Scottie Scheffler,17,6.4,2.21,2024-08-11,Wyndham Championship
1,Xander Schauffele,17,10.9,3.77,2024-08-11,Wyndham Championship
2,Alvaro Ortiz,1,13.0,18.76,2024-08-11,Wyndham Championship


### Course History

In [7]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

# Filter history_df for only the course we're targeting
target_course = tournament_config["new"]["course"]
course_df = history_df[history_df["COURSE"] == target_course]
course_hist = get_course_history("data/golf.db", course_df)

# View example
for date, df in course_hist.items():
    if not df.empty:
        print(f"\n🏌️‍♂️ Course history for {df['TOURNAMENT'].iloc[0]} on {date}")
        display(df.head(3))


🏌️‍♂️ Course history for Wyndham Championship on 2016-08-21


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,1,90.0,129.84,2016-08-21,Sedgefield Country Club,Wyndham Championship
1,Adam Hadwin,1,51.0,73.58,2016-08-21,Sedgefield Country Club,Wyndham Championship
2,Adam Scott,1,63.0,90.89,2016-08-21,Sedgefield Country Club,Wyndham Championship



🏌️‍♂️ Course history for Wyndham Championship on 2017-08-20


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,1,90.0,129.84,2017-08-20,Sedgefield Country Club,Wyndham Championship
1,Abraham Ancer,1,90.0,129.84,2017-08-20,Sedgefield Country Club,Wyndham Championship
2,Adam Hadwin,2,42.0,38.23,2017-08-20,Sedgefield Country Club,Wyndham Championship



🏌️‍♂️ Course history for Wyndham Championship on 2018-08-19


...

,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,5,73.8,41.19,2022-08-07,Sedgefield Country Club,Wyndham Championship
1,Aaron Wise,3,61.3,44.22,2022-08-07,Sedgefield Country Club,Wyndham Championship
2,Abraham Ancer,3,68.0,49.05,2022-08-07,Sedgefield Country Club,Wyndham Championship



🏌️‍♂️ Course history for Wyndham Championship on 2023-08-06


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,5,73.8,41.19,2023-08-06,Sedgefield Country Club,Wyndham Championship
1,Aaron Rai,1,71.0,102.43,2023-08-06,Sedgefield Country Club,Wyndham Championship
2,Aaron Wise,4,49.3,30.63,2023-08-06,Sedgefield Country Club,Wyndham Championship



🏌️‍♂️ Course history for Wyndham Championship on 2024-08-11


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,6,76.5,39.31,2024-08-11,Sedgefield Country Club,Wyndham Championship
1,Aaron Rai,2,80.5,73.27,2024-08-11,Sedgefield Country Club,Wyndham Championship
2,Aaron Wise,4,49.3,30.63,2024-08-11,Sedgefield Country Club,Wyndham Championship


In [8]:
#  DEBUGGING: Check for empty DataFrames or missing columns
for date_key, df in course_hist.items():
    if df.empty:
        print(f"ℹ️ Empty course_hist for date {date_key}")
    elif "PLAYER" not in df.columns:
        print(f"❌ Missing 'PLAYER' column in course_hist[{date_key}]. Columns present: {df.columns.tolist()}")
        display(df.head())

## Training Dataset

In [9]:
pd.set_option("display.max_columns", None)   # Show all columns
importlib.reload(utils.db_utils)
from utils.db_utils import build_training_rows
training_df = build_training_rows(
    db_path,
    history_df,
    cuts,
    recent_form,
    course_hist,
)
training_df.head(10)
# training_df.info()
# training_df.isna().sum().sort_values(ascending=False)

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Si Woo Kim,1,1,42.0,0.605,53.0,0.257,140.0,-0.168,9.0,0.403,118.0,-0.102,46.0,21.0,78.0,3.06,34.0,4.02,17.0,4.59,18.0,126.0,69.0,293.5,57.0,62.84,112.0,64.83,20.0,62.32,52.0,2.56,125.0,70.0,601.72,30.09,1.0,50.0,16.42,NaN,NaN,1
1,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Luke Donald,2,2,76.0,0.230,180.0,-0.665,27.0,0.465,4.0,0.429,68.0,0.148,170.0,17.9,50.0,3.04,48.0,4.03,146.0,4.73,180.0,295.0,151.0,282.9,144.0,56.69,118.0,64.61,6.0,64.15,69.0,2.04,100.0,62.5,490.68,30.67,0.0,57.4,20.26,26.0,37.51,1
2,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Hideki Matsuyama,T3,3,6.0,1.315,19.0,0.527,3.0,0.774,94.0,0.015,102.0,-0.025,6.0,23.52,22.0,3.02,14.0,4.01,17.0,4.59,21.0,132.0,65.0,294.5,67.0,62.04,35.0,68.11,46.0,60.86,18.0,4.46,20.0,61.5,1063.44,81.80,1.0,41.5,15.73,90.0,129.84,1
3,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Brandt Snedeker,T3,3,44.0,0.573,116.0,-0.059,78.0,0.164,18.0,0.320,25.0,0.395,32.0,21.41,78.0,3.06,2.0,3.98,100.0,4.69,99.0,188.0,75.0,292.6,113.0,58.69,90.0,65.67,16.0,62.73,20.0,4.08,25.0,72.2,1436.35,79.80,4.0,39.8,13.52,43.0,62.04,1
4,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Rafa Cabrera Bello,T5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,30.0,3.40,30.0,85.7,0.00,0.00,4.0,40.4,19.43,NaN,NaN,1
5,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Billy Horschel,T5,5,32.0,0.717,59.0,0.201,36.0,0.422,136.0,-0.102,77.0,0.112,48.0,20.95,146.0,3.10,48.0,4.03,100.0,4.69,52.0,156.0,72.0,293.0,84.0,60.71,45.0,67.7,166.0,54.55,71.0,2.02,80.0,84.6,514.28,39.56,1.0,39.1,14.82,90.0,129.84,1
6,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Graeme McDowell,T5,5,107.0,0.021,132.0,-0.129,87.0,0.153,104.0,-0.003,137.0,-0.199,108.0,19.72,78.0,3.06,136.0,4.07,100.0,4.69,68.0,169.0,156.0,281.4,13.0,67.7,125.0,64.44,86.0,58.85,80.0,1.82,100.0,46.2,482.43,37.11,0.0,58.2,22.05,NaN,NaN,1
7,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Brett Stegmaier,T5,5,93.0,0.109,70.0,0.128,93.0,0.133,152.0,-0.152,109.0,-0.061,121.0,19.48,160.0,3.11,48.0,4.03,50.0,4.64,21.0,132.0,52.0,295.3,80.0,60.93,60.0,66.8,97.0,58.27,248.0,0.73,NaN,50.0,188.79,10.49,0.0,69.3,23.54,NaN,NaN,1
8,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Johnson Wagner,T5,5,95.0,0.096,93.0,0.030,69.0,0.219,153.0,-0.154,41.0,0.273,64.0,20.54,50.0,3.04,34.0,4.02,68.0,4.66,78.0,177.0,101.0,289.4,76.0,61.13,13.0,69.38,109.0,57.81,166.0,1.02,NaN,58.8,334.29,19.66,1.0,59.9,20.72,90.0,129.84,1
9,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Bud Cauley,T10,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,277.0,0.65,80.0,70.0,319.98,32.00,5.0,47.8,19.93,NaN,NaN,1


In [10]:
# Play with different years to see if I notice anything wrong (check Odds for tournament name mismatches)
training_df[training_df["SEASON"] == 2024].head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
1247,2024,2024-08-11,Wyndham Championship,Sedgefield Country Club,Aaron Rai,1,1,9.0,1.131,36.0,0.303,8.0,0.676,56.0,0.152,83.0,0.054,99.0,22.10%,38.0,3.01,10.0,3.97,44.0,4.54,53.0,153.0,152.0,293.8,1.0,72.02%,7.0,71.58%,33.0,63.18%,21.0,3.2294,30.0,83.3,851.657,47.31,11.0,37.5,12.74,80.5,73.27,1
1248,2024,2024-08-11,Wyndham Championship,Sedgefield Country Club,Max Greyserman,2,2,93.0,0.076,66.0,0.161,110.0,-0.034,120.0,-0.051,5.0,0.625,38.0,23.74%,52.0,3.02,43.0,3.99,15.0,4.50,40.0,143.0,18.0,312.2,125.0,58.03%,80.0,67.31%,47.0,62.07%,52.0,2.0569,80.0,56.2,589.273,36.83,5.0,54.6,19.27,NaN,NaN,1
1249,2024,2024-08-11,Wyndham Championship,Sedgefield Country Club,J.J. Spaun,T3,3,47.0,0.458,109.0,-0.028,17.0,0.473,99.0,0.013,114.0,-0.080,76.0,22.62%,126.0,3.07,26.0,3.98,70.0,4.56,84.0,174.0,85.0,302.1,89.0,60.49%,18.0,70.56%,106.0,58.76%,117.0,1.1299,80.0,47.1,268.231,15.78,3.0,62.9,21.76,65.5,33.66,1
1250,2024,2024-08-11,Wyndham Championship,Sedgefield Country Club,Ryo Hisatsune,T3,3,59.0,0.384,91.0,0.056,47.0,0.282,89.0,0.046,117.0,-0.084,77.0,22.59%,74.0,3.04,26.0,3.98,129.0,4.63,79.0,171.0,125.0,297.8,46.0,63.83%,21.0,70.37%,134.0,57.29%,88.0,1.3308,200.0,57.9,332.308,17.49,0.0,58.0,19.36,NaN,NaN,1
1251,2024,2024-08-11,Wyndham Championship,Sedgefield Country Club,Luke Clanton,5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,148.0,0.9236,60.0,75.0,0.000,0.00,0.0,35.8,22.24,NaN,NaN,1


### Normalization
***Fix Historical Odds***

Many names do not have historical odds but rather are part of the "field." So giving these "NaN" values the average odds (as I would for missing stats) isn't a good approximation of reality – they are typically the "field" because they are not notable and have poor odds individually.

There are also some crazy odds numbers occasionally (1000/1 or 3000/1) that don't happen all that much and are making that tail too long.

This cell assigns odds of 1000/1 for anyone missing and clips anything larger to 1000/1 to normalize the data better and assign the missing odds more appropriately where they belong.

***OWGR Adjustment***

The NaN's for OWGR are similarly bad players.  So we should assign these the worst ranking instead of the mean and clip it at 1000 just in case there are ever outliers.

For the OWGR score, it should match the lowest score in the dataset to assign that person the same as the worst.

***Recent Form Adjustment***

The NaN's for Recent Form mean that this player has not played any tournaments in the lookback period (9 months).  Similar to Odds and OWGR, I want to punish those that don't play often in my model.  Set these to 90 (i.e. like they miss a lot of cuts - because if they aren't playing they aren't good enough to make it to these tournaments very frequently).  The adj_form feature will also need to be updated here with the new data.

***FedEx Cup Point Adjustment***

The NaN's for FedEx Cup Points means there is no data.  Therefore they should be 0.

In [11]:
# === VEGAS ODDS ===
training_df["VEGAS_ODDS"] = training_df["VEGAS_ODDS"].fillna(1000).clip(upper=1000)

# === OWGR and OWGR_RANK ===
owgr_min = training_df["OWGR"].min(skipna=True)
training_df["OWGR"] = training_df["OWGR"].fillna(owgr_min)
training_df["OWGR_RANK"] = training_df["OWGR_RANK"].fillna(1000).astype(float).clip(upper=1000)

# === RECENT FORM and adj_form ===
training_df["RECENT_FORM"] = training_df["RECENT_FORM"].fillna(90)
if "TOTAL_EVENTS_PLAYED" in training_df.columns:
    training_df["adj_form"] = (
        training_df["RECENT_FORM"] / np.log1p(training_df["TOTAL_EVENTS_PLAYED"])
    ).round(2)

# === FEDEX CUP POINTS ===
training_df["FEDEX_CUP_POINTS"] = training_df["FEDEX_CUP_POINTS"].fillna(0)

training_df.head(5)


,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Si Woo Kim,1,1,42.0,0.605,53.0,0.257,140.0,-0.168,9.0,0.403,118.0,-0.102,46.0,21.0,78.0,3.06,34.0,4.02,17.0,4.59,18.0,126.0,69.0,293.5,57.0,62.84,112.0,64.83,20.0,62.32,52.0,2.56,125.0,70.0,601.72,30.09,1.0,50.0,16.42,NaN,NaN,1
1,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Luke Donald,2,2,76.0,0.230,180.0,-0.665,27.0,0.465,4.0,0.429,68.0,0.148,170.0,17.9,50.0,3.04,48.0,4.03,146.0,4.73,180.0,295.0,151.0,282.9,144.0,56.69,118.0,64.61,6.0,64.15,69.0,2.04,100.0,62.5,490.68,30.67,0.0,57.4,20.26,26.0,37.51,1
2,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Hideki Matsuyama,T3,3,6.0,1.315,19.0,0.527,3.0,0.774,94.0,0.015,102.0,-0.025,6.0,23.52,22.0,3.02,14.0,4.01,17.0,4.59,21.0,132.0,65.0,294.5,67.0,62.04,35.0,68.11,46.0,60.86,18.0,4.46,20.0,61.5,1063.44,81.80,1.0,41.5,15.73,90.0,129.84,1
3,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Brandt Snedeker,T3,3,44.0,0.573,116.0,-0.059,78.0,0.164,18.0,0.320,25.0,0.395,32.0,21.41,78.0,3.06,2.0,3.98,100.0,4.69,99.0,188.0,75.0,292.6,113.0,58.69,90.0,65.67,16.0,62.73,20.0,4.08,25.0,72.2,1436.35,79.80,4.0,39.8,13.52,43.0,62.04,1
4,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Rafa Cabrera Bello,T5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,30.0,3.40,30.0,85.7,0.00,0.00,4.0,40.4,19.43,NaN,NaN,1


#### Average the NaNs
At this point, the intentional offsetting of NaNs for poor players is over.  I don't want to penalize statistics or course history because you never know where they might land on the spectrum, so for these NaN values, we will take the mean.

In [12]:
# === Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in training_df.columns:
        training_df[col] = (
            training_df[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
            .astype(float)
        )

# Select only the numeric columns
numeric_columns = training_df.select_dtypes(include=['float64', 'int64'])

# Fill NaN values with the mean of each column
numeric_columns = numeric_columns.fillna(numeric_columns.mean())

# Update the original dataframe with the filled numeric columns
training_df.update(numeric_columns)

training_df.head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Si Woo Kim,1,1,42.000000,0.605000,53.000000,0.25700,140.000000,-0.168000,9.000000,0.40300,118.000000,-0.10200,46.000000,21.000000,78.000000,3.060000,34.000000,4.020000,17.000000,4.590000,18.000000,126.000000,69.000000,293.500000,57.00000,62.840000,112.000000,64.830000,20.000000,62.320000,52.0,2.56,125.0,70.0,601.72,30.09,1.0,50.0,16.42,59.818304,58.687842,1
1,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Luke Donald,2,2,76.000000,0.230000,180.000000,-0.66500,27.000000,0.465000,4.000000,0.42900,68.000000,0.14800,170.000000,17.900000,50.000000,3.040000,48.000000,4.030000,146.000000,4.730000,180.000000,295.000000,151.000000,282.900000,144.00000,56.690000,118.000000,64.610000,6.000000,64.150000,69.0,2.04,100.0,62.5,490.68,30.67,0.0,57.4,20.26,26.000000,37.510000,1
2,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Hideki Matsuyama,T3,3,6.000000,1.315000,19.000000,0.52700,3.000000,0.774000,94.000000,0.01500,102.000000,-0.02500,6.000000,23.520000,22.000000,3.020000,14.000000,4.010000,17.000000,4.590000,21.000000,132.000000,65.000000,294.500000,67.00000,62.040000,35.000000,68.110000,46.000000,60.860000,18.0,4.46,20.0,61.5,1063.44,81.80,1.0,41.5,15.73,90.000000,129.840000,1
3,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Brandt Snedeker,T3,3,44.000000,0.573000,116.000000,-0.05900,78.000000,0.164000,18.000000,0.32000,25.000000,0.39500,32.000000,21.410000,78.000000,3.060000,2.000000,3.980000,100.000000,4.690000,99.000000,188.000000,75.000000,292.600000,113.00000,58.690000,90.000000,65.670000,16.000000,62.730000,20.0,4.08,25.0,72.2,1436.35,79.80,4.0,39.8,13.52,43.000000,62.040000,1
4,2016,2016-08-21,Wyndham Championship,Sedgefield Country Club,Rafa Cabrera Bello,T5,5,108.402367,-0.045962,105.005917,-0.03122,105.649197,-0.017095,100.447168,0.00197,101.153001,-0.00735,105.448859,20.837117,92.592561,3.058166,97.351648,4.034683,99.732037,4.644387,101.645816,196.696534,103.422654,295.711496,93.27388,60.854565,97.868132,66.233711,100.522401,58.743043,30.0,3.40,30.0,85.7,0.00,0.00,4.0,40.4,19.43,59.818304,58.687842,1


In [13]:
# Check for NaN values
assert training_df.isna().sum().sum() == 0, "🚨 Still missing values!"

In [14]:
# Inspect which columns have NaN values
training_df.isna().sum().sort_values(ascending=False).head(10)


SEASON         0
ENDING_DATE    0
TOURNAMENT     0
COURSE         0
PLAYER         0
POS            0
FINAL_POS      0
SGTTG_RANK     0
SGTTG          0
SGOTT_RANK     0
dtype: int64

### Check Features for Outliers and General Health (Histograms)
Now that all the data is cleaned, let's look at the distribution graphs.

In [19]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Select numeric columns with >1 unique value, excluding label
df_numeric = training_df.select_dtypes(include=np.number)
df_numeric = df_numeric.loc[:, df_numeric.nunique() > 1]
df_numeric = df_numeric.drop(columns=["TOP_20"], errors="ignore")

# Create subplots
n_cols = 3
n_rows = int(np.ceil(len(df_numeric.columns) / n_cols))
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=df_numeric.columns)

# Add one histogram per feature with smart binning
for i, column in enumerate(df_numeric.columns):
    row = i // n_cols + 1
    col = i % n_cols + 1

    # Compute dynamic bin count
    col_range = df_numeric[column].max() - df_numeric[column].min()
    n_unique = df_numeric[column].nunique()
    nbins = min(50, max(5, int(n_unique / 2))) if col_range < 10 else 50  # tighter for short-range

    fig.add_trace(
        go.Histogram(
            x=df_numeric[column],
            nbinsx=nbins,
            histnorm="density",
            marker=dict(color="#636EFA"),
            opacity=0.8
        ),
        row=row,
        col=col
    )

# Style the figure
fig.update_layout(
    height=300 * n_rows,
    width=1000,
    title_text="Feature Distributions (with Adaptive Binning)",
    template="plotly_dark",
    showlegend=False
)

fig.show()

# Current Week Data

## Import 2025 Field
**Important!**  Save curren't week's `DKSalaries.csv` into Data folder before this step.

In [15]:
importlib.reload(utils.db_utils)
from utils.db_utils import DK_PLAYER_NAME_MAP
from utils.db_utils import standardize_player_names

# Load DraftKings player list
dk = pd.read_csv("data/DKSalaries.csv", usecols=["Name", "Salary"])

# Normalize DK names to PGA naming convention
dk["Name"] = dk["Name"].replace(DK_PLAYER_NAME_MAP)

# Rename for consistency
dk = dk.rename(columns={"Name": "PLAYER", "Salary": "SALARY"})

dk = standardize_player_names(dk)  # ✅ Normalize to match DB

dk.head(10)

,PLAYER,SALARY
0,Matt Fitzpatrick,10600
1,Keegan Bradley,10300
2,Ben Griffin,10100
3,Hideki Matsuyama,9800
4,Robert Macintyre,9600
5,Jordan Spieth,9400
6,Aaron Rai,9300
7,Harry Hall,9200
8,Jake Knapp,9100
9,Akshay Bhatia,9000


## Vegas Odds
Confirm the [Vegas Odds](http://golfodds.com/weekly-odds.html) page is updated with current tournament.

Note: sometimes this site can give errors and not load correctly.  Just have to do it again later.
Another trick to try is change the url try to run it and then put the url back the way it was.  Worked once, maybe random coincidence.

In [16]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_current_week_odds

odds_current = get_current_week_odds(season=2025, tournament_name=new_tournament_name)
odds_current.head(10)


,SEASON,TOURNAMENT,PLAYER,ODDS,VEGAS_ODDS
1,2025,Wyndham Championship,Matt Fitzpatrick,18/1,18.0
2,2025,Wyndham Championship,Aaron Rai,20/1,20.0
3,2025,Wyndham Championship,Keegan Bradley,25/1,25.0
4,2025,Wyndham Championship,Ben Griffin,25/1,25.0
5,2025,Wyndham Championship,Robert MacIntyre,30/1,30.0
6,2025,Wyndham Championship,Si Woo Kim,30/1,30.0
7,2025,Wyndham Championship,Jordan Spieth,35/1,35.0
8,2025,Wyndham Championship,Hideki Matsuyama,35/1,35.0
9,2025,Wyndham Championship,Lucas Glover,35/1,35.0
10,2025,Wyndham Championship,Jake Knapp,35/1,35.0


## Cut Percentage and FedEx Points

In [17]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts_rolling = get_cut_and_fedex_history(db_path, this_week_history, window_months=9)
cuts_current = cuts_rolling[this_week_key].copy()
cuts_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.000,0.0,0.00,0,2025-08-03,Wyndham Championship
1,Aaron Baddeley,15,6,49.088,40.0,3.27,0,2025-08-03,Wyndham Championship
2,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2025-08-03,Wyndham Championship
3,Aaron Rai,19,15,695.647,78.9,36.61,4,2025-08-03,Wyndham Championship
4,Aaron Wise,4,1,6.333,25.0,1.58,0,2025-08-03,Wyndham Championship
5,Adam Hadwin,21,14,223.988,66.7,10.67,1,2025-08-03,Wyndham Championship
6,Adam Long,1,0,0.000,0.0,0.00,0,2025-08-03,Wyndham Championship
7,Adam Schenk,20,7,258.004,35.0,12.90,0,2025-08-03,Wyndham Championship
8,Adam Scott,16,13,445.528,81.2,27.85,1,2025-08-03,Wyndham Championship
9,Adam Svensson,19,10,149.843,52.6,7.89,1,2025-08-03,Wyndham Championship


## Recent Form

In [18]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish(db_path, this_week_history, window_months=9)
recent_form_current = recent_form[this_week_key].copy()

recent_form_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Scottie Scheffler,17,6.6,2.28,2025-08-03,Wyndham Championship
1,Jason Scrivener,1,8.0,11.54,2025-08-03,Wyndham Championship
2,Bubba Watson,1,14.0,20.20,2025-08-03,Wyndham Championship
3,Jon Rahm,4,15.8,9.82,2025-08-03,Wyndham Championship
4,Rory McIlroy,13,16.8,6.37,2025-08-03,Wyndham Championship
5,Robin Williams,1,17.0,24.53,2025-08-03,Wyndham Championship
6,Andy Sullivan,1,17.0,24.53,2025-08-03,Wyndham Championship
7,Joel Girrbach,1,20.0,28.85,2025-08-03,Wyndham Championship
8,Kensei Hirata,1,21.0,30.30,2025-08-03,Wyndham Championship
9,Kristoffer Reitan,2,21.5,19.57,2025-08-03,Wyndham Championship


## Course History

In [19]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

course_hist = get_course_history(db_path, this_week_history, lookback_years=7)
course_hist_current = course_hist[this_week_key].copy()

course_hist_current.head(10)


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,6,79.0,40.60,2025-08-03,Sedgefield Country Club,Wyndham Championship
1,Aaron Rai,3,54.0,38.95,2025-08-03,Sedgefield Country Club,Wyndham Championship
2,Aaron Wise,4,49.3,30.63,2025-08-03,Sedgefield Country Club,Wyndham Championship
3,Abraham Ancer,2,57.0,51.88,2025-08-03,Sedgefield Country Club,Wyndham Championship
4,Adam Hadwin,4,54.5,33.86,2025-08-03,Sedgefield Country Club,Wyndham Championship
5,Adam Long,5,78.2,43.64,2025-08-03,Sedgefield Country Club,Wyndham Championship
6,Adam Schenk,7,80.7,38.81,2025-08-03,Sedgefield Country Club,Wyndham Championship
7,Adam Scott,3,28.3,20.41,2025-08-03,Sedgefield Country Club,Wyndham Championship
8,Adam Svensson,4,30.8,19.14,2025-08-03,Sedgefield Country Club,Wyndham Championship
9,Adrien Dumont de Chassart,1,90.0,129.84,2025-08-03,Sedgefield Country Club,Wyndham Championship


## Merged Dataframe

In [20]:
importlib.reload(utils.db_utils)
from utils.db_utils import build_test_rows

this_week = build_test_rows(
    db_path=db_path,
    stats_df=stats_df,
    odds_df=odds_current,
    cuts_df=cuts_current,
    recent_form_df=recent_form_current,
    course_hist_df=course_hist_current,
    dk_df=dk,
    season=tournament_config["new"]["season"]
)

this_week.head()

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Aaron Baddeley,173.0,-1.327,175.0,-1.232,169.0,-0.543,9.0,0.447,158.0,-0.422,176.0,17.48%,135.0,3.09,164.0,4.09,176.0,4.78,173.0,284,164.0,289.8,120.0,57.27%,175.0,59.76%,85.0,59.60%,514.0,.2509,2025.0,2000.0,40.0,49.088,3.27,0.0,77.6,27.99,79.0,40.60,6000
1,Aaron Rai,29.0,0.724,22.0,0.390,57.0,0.246,64.0,0.087,108.0,-0.047,108.0,21.35%,176.0,3.18,13.0,3.98,38.0,4.56,74.0,170,168.0,288.6,2.0,73.37%,23.0,69.53%,93.0,59.26%,35.0,2.5936,2025.0,20.0,78.9,695.647,36.61,4.0,38.6,12.88,54.0,38.95,9300
2,Aaron Wise,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6000
3,Adam Hadwin,160.0,-0.675,135.0,-0.197,163.0,-0.462,101.0,-0.016,84.0,0.071,76.0,22.06%,39.0,3.03,134.0,4.05,128.0,4.64,91.0,178,107.0,300.3,71.0,61.22%,147.0,63.68%,161.0,55.02%,141.0,1.0540,2025.0,250.0,66.7,223.988,10.67,1.0,62.2,20.12,54.5,33.86,6400
4,Adam Schenk,128.0,-0.185,102.0,0.009,122.0,-0.102,129.0,-0.093,130.0,-0.135,122.0,20.96%,39.0,3.03,162.0,4.08,72.0,4.59,148.0,213,57.0,305.9,156.0,53.93%,167.0,61.74%,117.0,57.81%,184.0,.7831,2025.0,400.0,35.0,258.004,12.90,0.0,70.0,22.99,80.7,38.81,6200


### Dataframe Normalization

Run the same normalization techniques that I ran on the historical data:
- Any NaN Odds go to 1000/1 and clip all values at 1000/1
- OWGR NaN values go to 1000
- Recent form NaN values act like a MC at 90
- Average everything else.

In [21]:
# === 1. Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in this_week.columns:
        this_week[col] = (
            this_week[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
        )
        this_week[col] = pd.to_numeric(this_week[col], errors="coerce")

# === 2. Normalize & assign fallback values for key fields ===
this_week["VEGAS_ODDS"] = pd.to_numeric(this_week["VEGAS_ODDS"], errors="coerce").fillna(1000).clip(upper=1000)

if "OWGR" in this_week.columns:
    this_week["OWGR"] = pd.to_numeric(this_week["OWGR"], errors="coerce")
    max_owgr = this_week["OWGR"].dropna().max()
    this_week["OWGR"] = this_week["OWGR"].fillna(max_owgr).clip(upper=1000)

if "OWGR_RANK" in this_week.columns:
    this_week["OWGR_RANK"] = pd.to_numeric(this_week["OWGR_RANK"], errors="coerce").fillna(1000).clip(upper=1000)

this_week["RECENT_FORM"] = pd.to_numeric(this_week["RECENT_FORM"], errors="coerce").fillna(90)
this_week["FEDEX_CUP_POINTS"] = pd.to_numeric(this_week["FEDEX_CUP_POINTS"], errors="coerce").fillna(0)

if "COURSE_HISTORY" in this_week.columns:
    this_week["COURSE_HISTORY"] = pd.to_numeric(this_week["COURSE_HISTORY"], errors="coerce")
    ch_mean = this_week["COURSE_HISTORY"].mean()
    this_week["COURSE_HISTORY"] = this_week["COURSE_HISTORY"].fillna(ch_mean)

# === 3. Fill all remaining NaNs in numeric columns ===
# Recast everything that might look numeric to be sure
for col in this_week.columns:
    if col not in ["PLAYER", "SALARY", "TOURNAMENT", "SEASON"]:
        try:
            this_week[col] = pd.to_numeric(this_week[col], errors="coerce")
        except Exception:
            continue

numeric_cols = this_week.select_dtypes(include=["number"]).columns.tolist()

for col in numeric_cols:
    if this_week[col].isna().any():
        col_mean = this_week[col].mean()
        this_week[col] = this_week[col].fillna(col_mean if not np.isnan(col_mean) else 0)


In [22]:
# === Final Check: No NaNs Should Remain ===
assert this_week.isna().sum().sum() == 0, "🚨 Still missing values in prediction set!"

In [23]:
# === Check for Remaining Missing Values ===
missing_summary = this_week.isna().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

print("🧹 Columns still containing NaN values:")
display(missing_summary)

🧹 Columns still containing NaN values:


Series([], dtype: int64)

In [24]:
this_week.head(10)

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Aaron Baddeley,173.000000,-1.327000,175.000000,-1.232000,169.000000,-0.543,9.000000,0.447000,158.0,-0.422000,176.000000,17.480000,135.000000,3.090000,164.000000,4.090000,176.000000,4.780,173.000000,284.000000,164.000000,289.8,120.000000,57.270000,175.000000,59.760,85.000000,59.600000,514.0,0.2509,2025.0,1000.0,40.000000,49.088,3.270000,0.000000,77.6,27.990000,79.000000,40.600000,6000
1,Aaron Rai,29.000000,0.724000,22.000000,0.390000,57.000000,0.246,64.000000,0.087000,108.0,-0.047000,108.000000,21.350000,176.000000,3.180000,13.000000,3.980000,38.000000,4.560,74.000000,170.000000,168.000000,288.6,2.000000,73.370000,23.000000,69.530,93.000000,59.260000,35.0,2.5936,2025.0,20.0,78.900000,695.647,36.610000,4.000000,38.6,12.880000,54.000000,38.950000,9300
2,Aaron Wise,95.769231,0.031577,94.976923,-0.022077,93.046154,0.028,87.869231,0.025638,93.3,0.000708,96.523077,21.574538,84.638462,3.058538,84.392308,4.024769,89.661538,4.608,90.776923,178.269231,91.907692,302.0,86.361538,60.114769,93.453846,66.098,86.853846,59.581538,1000.0,4.8821,2025.0,1000.0,57.046429,0.000,22.782857,1.578571,90.0,20.407357,57.112264,48.395472,6000
3,Adam Hadwin,160.000000,-0.675000,135.000000,-0.197000,163.000000,-0.462,101.000000,-0.016000,84.0,0.071000,76.000000,22.060000,39.000000,3.030000,134.000000,4.050000,128.000000,4.640,91.000000,178.000000,107.000000,300.3,71.000000,61.220000,147.000000,63.680,161.000000,55.020000,141.0,1.0540,2025.0,250.0,66.700000,223.988,10.670000,1.000000,62.2,20.120000,54.500000,33.860000,6400
4,Adam Schenk,128.000000,-0.185000,102.000000,0.009000,122.000000,-0.102,129.000000,-0.093000,130.0,-0.135000,122.000000,20.960000,39.000000,3.030000,162.000000,4.080000,72.000000,4.590,148.000000,213.000000,57.000000,305.9,156.000000,53.930000,167.000000,61.740,117.000000,57.810000,184.0,0.7831,2025.0,400.0,35.000000,258.004,12.900000,0.000000,70.0,22.990000,80.700000,38.810000,6200
5,Adam Scott,41.000000,0.548000,32.000000,0.312000,73.000000,0.157,67.000000,0.079000,115.0,-0.069000,107.000000,21.360000,147.000000,3.100000,146.000000,4.060000,34.000000,4.550,131.000000,201.000000,79.000000,303.8,122.000000,56.930000,115.000000,65.230,142.000000,56.750000,45.0,2.3495,2025.0,40.0,81.200000,445.528,27.850000,1.000000,42.6,15.040000,28.300000,20.410000,8600
6,Adam Svensson,63.000000,0.352000,130.000000,-0.177000,45.000000,0.278,28.000000,0.251000,168.0,-0.533000,154.000000,20.130000,54.000000,3.040000,117.000000,4.040000,38.000000,4.560,97.000000,181.000000,154.000000,295.1,27.000000,65.170000,41.000000,68.580,90.000000,59.420000,211.0,0.6908,2025.0,100.0,52.600000,149.843,7.890000,1.000000,65.5,21.860000,30.800000,19.140000,6900
7,Akshay Bhatia,74.000000,0.235000,114.000000,-0.027000,17.000000,0.559,159.000000,-0.297000,16.0,0.410000,12.000000,24.590000,11.000000,2.990000,41.000000,4.000000,133.000000,4.650,121.000000,195.000000,140.000000,296.7,55.000000,62.080000,60.000000,67.570,166.000000,54.160000,38.0,2.5468,2025.0,40.0,80.000000,908.990,45.450000,4.000000,38.7,12.710000,90.000000,55.920000,9000
8,Alex Noren,95.769231,0.031577,94.976923,-0.022077,93.046154,0.028,87.869231,0.025638,93.3,0.000708,96.523077,21.574538,84.638462,3.058538,84.392308,4.024769,89.661538,4.608,90.776923,178.269231,91.907692,302.0,86.361538,60.114769,93.453846,66.098,86.853846,59.581538,85.0,1.6740,2025.0,80.0,62.500000,219.250,27.410000,1.000000,51.8,23.580000,49.000000,44.600000,7200
9,Alex Smalley,25.000000,0.773000,20.000000,0.418000,79.000000,0.131,32.000000,0.224000,28.0,0.319000,18.000000,24.070000,54.00000

## Correlations
**Last 4 Years**

In [25]:
import plotly.express as px

# Get the 4 most recent seasons (or fewer if not available)
available_seasons = sorted(training_df["SEASON"].unique())[-4:]

# Dictionary to store correlation results
correlation_frames = {}

# Calculate correlations for each season
for season in available_seasons:
    df = training_df[training_df["SEASON"] == season].copy()

    # Numeric features only, drop final result
    numeric = df.select_dtypes(include=["float64", "int64"]).drop(columns=["FINAL_POS", "SEASON"], errors="ignore")

    # Skip if TOP_20 is not present
    if "TOP_20" not in numeric.columns:
        continue

    # Correlation with TOP_20
    cor = numeric.corr().abs()["TOP_20"].drop("TOP_20", errors="ignore").sort_values(ascending=False)
    cor_df = cor.reset_index()
    cor_df.columns = ["Feature", "Correlation"]
    correlation_frames[season] = cor_df

# Plot one bar chart per season
for season, cor_df in correlation_frames.items():
    fig = px.bar(
        cor_df,
        x="Feature",
        y="Correlation",
        title=f"🔎 Correlation to TOP_20 — Season {int(season)}",
        height=500,
        width=1000
    )

    fig.update_layout(
        xaxis_title="Feature",
        yaxis_title="Absolute Correlation to TOP_20",
        template="plotly_dark",
        xaxis_tickangle=-45
    )

    fig.show()


**Average of Last 4 Years**

In [26]:
import pandas as pd
import numpy as np
import plotly.express as px

# === Rebuild correlation matrices for the 4 most recent seasons ===
cor = {}
recent_seasons = sorted(training_df["SEASON"].unique())[-4:]

for season in recent_seasons:
    df_season = training_df[training_df["SEASON"] == season].copy()
    numeric_cols = df_season.select_dtypes(include=["float64", "int64"])
    cor[int(season)] = numeric_cols.corr().abs()

# === Combine into one summary DataFrame ===
cor_df = pd.DataFrame()

for season in recent_seasons:
    df = cor[season].reset_index()
    if "TOP_20" not in df.columns:
        continue
    temp = df[["index", "TOP_20"]].copy()
    temp = temp.rename(columns={"TOP_20": str(season)})
    if cor_df.empty:
        cor_df = temp
    else:
        cor_df = cor_df.merge(temp, on="index", how="outer")

# Drop target and metadata rows
cor_df = cor_df[~cor_df["index"].isin(["TOP_20", "FINAL_POS", "SEASON"])]

# Compute average correlation across seasons
cor_df["AVERAGE"] = cor_df.drop(columns=["index"]).mean(axis=1)
cor_df = cor_df.sort_values(by="AVERAGE", ascending=False)

# === Plotly Bar Chart ===
fig = px.bar(
    cor_df,
    x="index",
    y="AVERAGE",
    title="Average Correlation to TOP_20 (Last 4 Seasons)",
    labels={"index": "Feature", "AVERAGE": "Avg Correlation"},
    template="plotly_dark"
)

fig.update_layout(
    xaxis_tickangle=-45,
    height=600,
    width=1000,
    showlegend=False
)

fig.show()


# ML Model

## Predictors

In [27]:
# === Define Features and Target ===
target_col = "TOP_20"

# Columns to exclude from features
exclude = [
    "PLAYER", "TOURNAMENT", "COURSE", "ENDING_DATE", "SEASON", "TOURN_ID",
    "TOP_20", "FINAL_POS"  
]

# Select numeric feature columns
feature_cols = [
    col for col in training_df.columns
    if col not in exclude and training_df[col].dtype in [np.float64, np.int64]
]

print(f"Selected {len(feature_cols)} features:\n", feature_cols)

Selected 39 features:
 ['SGTTG_RANK', 'SGTTG', 'SGOTT_RANK', 'SGOTT', 'SGAPR_RANK', 'SGAPR', 'SGATG_RANK', 'SGATG', 'SGP_RANK', 'SGP', 'BIRDIES_RANK', 'BIRDIES', 'PAR_3_RANK', 'PAR_3', 'PAR_4_RANK', 'PAR_4', 'PAR_5_RANK', 'PAR_5', 'TOTAL_DRIVING_RANK', 'TOTAL_DRIVING', 'DRIVING_DISTANCE_RANK', 'DRIVING_DISTANCE', 'DRIVING_ACCURACY_RANK', 'DRIVING_ACCURACY', 'GIR_RANK', 'GIR', 'SCRAMBLING_RANK', 'SCRAMBLING', 'OWGR_RANK', 'OWGR', 'VEGAS_ODDS', 'CUT_PERCENTAGE', 'FEDEX_CUP_POINTS', 'form_density', 'CONSECUTIVE_CUTS', 'RECENT_FORM', 'adj_form', 'COURSE_HISTORY', 'adj_ch']


### Random Forest
Initial default model before hyperparameter tuning.

This one actually performed really well (ROC AUC above 0.85).  Strangely doing hyperparameter tuning hurt the model (less than 0.7), so I removed that step and we will just use this default model.  This does cross-validation testing on 5 folds to ensure robust train/test splits.

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# === Standardize features ===
scaler = StandardScaler()
X_rf_std = scaler.fit_transform(training_df[feature_cols])
y_rf = training_df["TOP_20"]

# === Balance the dataset with SMOTE and undersampling ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
pipeline = Pipeline([("o", over), ("u", under)])
X_rf_bal, y_rf_bal = pipeline.fit_resample(X_rf_std, y_rf)

# === Define and evaluate Random Forest ===
rf = RandomForestClassifier(n_estimators=100, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X_rf_bal, y_rf_bal, cv=cv, scoring="roc_auc")

print("📊 Random Forest (All Features)")
print("Cross-validation scores:", scores)
print("Average ROC AUC score: {:.2f}".format(scores.mean()))


📊 Random Forest (All Features)
Cross-validation scores: [0.94505157 0.95123284 0.93610537 0.94148006 0.95620863]
Average ROC AUC score: 0.95


Random Forest Training & Feature Importance

Now we train on all the data (no longer cross-validation folds) and extract the feature importance to see how it compares with linear correlations.


In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import plotly.express as px

# === Resample the full dataset ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
resample_pipeline = Pipeline([("o", over), ("u", under)])
X_all_resampled, y_all_resampled = resample_pipeline.fit_resample(
    training_df[feature_cols], training_df["TOP_20"]
)

# === Standardize ===
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all_resampled)

# === Train Calibrated Random Forest ===
base_rf = RandomForestClassifier(random_state=42)
final_model = CalibratedClassifierCV(estimator=base_rf, method="sigmoid", cv=5)
final_model.fit(X_all_scaled, y_all_resampled)

# === Aggregate feature importances across all CV folds ===
all_importances = np.array([
    est.estimator.feature_importances_ for est in final_model.calibrated_classifiers_
])
mean_importance = all_importances.mean(axis=0)

importance_df = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": mean_importance
}).sort_values(by="Importance", ascending=False)

# === Plot feature importances ===
fig = px.bar(
    importance_df,
    x="Feature",
    y="Importance",
    title="Random Forest Feature Importances",
    template="plotly_dark"
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


Use this model on this week's data to predict the best performers.

In [30]:
# === Prepare test data (this week's players) ===

# Select and standardize features using the same scaler as training
X_test = this_week[feature_cols]
X_test_scaled = scaler.transform(X_test)

# === Predict probabilities for Top 20 finish ===
# CalibratedClassifierCV returns well-calibrated probabilities
this_week["PROBABILITY"] = final_model.predict_proba(X_test_scaled)[:, 1]

# === Sort results by highest predicted probability ===
this_week_sorted = this_week.sort_values(by="PROBABILITY", ascending=False).reset_index(drop=True)

# === Define final column order explicitly ===
columns_to_show = [
    "PLAYER", "SALARY", "PROBABILITY",
    "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", "CUT_PERCENTAGE", 
    "OWGR_RANK", "FEDEX_CUP_POINTS",

    # SG stats
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",

    # Scoring/Performance
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",

    # Driving / Ball Striking
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Keep only columns that exist in the dataframe
columns_to_show = [col for col in columns_to_show if col in this_week_sorted.columns]

# Final dataframe for display/export
export_df = this_week_sorted[columns_to_show].copy()

# Preview top 20
export_df.head(20)


,PLAYER,SALARY,PROBABILITY,RECENT_FORM,COURSE_HISTORY,VEGAS_ODDS,CUT_PERCENTAGE,OWGR_RANK,FEDEX_CUP_POINTS,SGTTG_RANK,SGOTT_RANK,SGAPR_RANK,SGATG_RANK,SGP_RANK,BIRDIES_RANK,PAR_3_RANK,PAR_4_RANK,PAR_5_RANK,TOTAL_DRIVING_RANK,DRIVING_DISTANCE_RANK,DRIVING_ACCURACY_RANK,GIR_RANK,SCRAMBLING_RANK
0,Si Woo Kim,8700,0.844904,47.8,44.700000,30.0,68.000000,67.0,881.749,12.000000,40.000000,39.000000,13.000000,150.0,38.000000,121.000000,41.000000,18.000000,76.000000,144.000000,28.000000,77.000000,60.000000
1,Jake Knapp,9100,0.760719,42.2,90.000000,35.0,76.200000,77.0,769.686,69.000000,63.000000,80.000000,105.000000,11.0,4.000000,90.000000,7.000000,4.000000,76.000000,21.000000,151.000000,43.000000,21.000000
2,Kurt Kitayama,8800,0.737553,52.4,90.000000,40.0,61.100000,39.0,816.559,8.000000,12.000000,36.000000,48.000000,152.0,5.000000,90.000000,10.000000,14.000000,39.000000,7.000000,139.000000,49.000000,84.000000
3,Ryan Fox,7300,0.611685,47.7,90.000000,80.0,73.300000,32.0,823.200,37.000000,97.000000,25.000000,65.000000,38.0,40.000000,103.000000,85.000000,38.000000,141.000000,53.000000,155.000000,126.000000,151.000000
4,Emiliano Grillo,7500,0.565220,54.9,59.000000,70.0,66.700000,99.0,586.013,77.000000,67.000000,40.000000,157.000000,40.0,84.000000,75.000000,41.000000,116.000000,16.000000,99.000000,16.000000,74.000000,112.000000
5,Keith Mitchell,7500,0.559763,49.4,50.400000,80.0,68.400000,101.0,478.759,44.000000,17.000000,55.000000,154.000000,93.0,6.000000,103.000000,13.000000,7.000000,11.000000,11.000000,94.000000,23.000000,88.000000
6,Hideki Matsuyama,9800,0.520358,35.6,70.300000,35.0,83.300000,12.0,1264.979,19.000000,128.000000,24.000000,5.000000,75.0,42.000000,29.000000,23.000000,105.000000,163.000000,115.000000,132.000000,89.000000,6.000000
7,Akshay Bhatia,9000,0.509032,38.7,90.000000,40.0,80.000000,38.0,908.990,74.000000,114.000000,17.000000,159.000000,16.0,12.000000,11.000000,41.000000,133.000000,121.000000,140.000000,55.000000,60.000000,166.000000
8,Lee Hodges,6900,0.474565,48.0,68.500000,125.0,64.700000,127.0,477.614,36.000000,91.000000,19.000000,94.000000,88.0,45.000000,39.000000,13.000000,7.000000,19.000000,76.000000,45.000000,16.000000,19.000000
9,Aaron Rai,9300,0.469283,38.6,54.000000,20.0,78.900000,35.0,695.647,29.000000,22.000000,57.000000,64.000000,108.0,108.000000,176.000000,13.000000,38.000000,74.000000,168.000000,2.000000,23.000000,93.000000


### Save to CSV

In [31]:
# Ensure required columns for spreadsheet export
if "COURSE_HISTORY" not in export_df.columns:
    export_df["COURSE_HISTORY"] = "-"

# Define columns to round (FedEx + stats)
columns_to_round = [
    "FEDEX_CUP_POINTS",
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

columns_to_round_1 = [
    "COURSE_HISTORY", "CUT_PERCENTAGE"
]

# Round only the selected columns to 0 decimal places
export_df[columns_to_round] = export_df[columns_to_round].round(0).astype("Int64")
# Round only the selected columns to 1 decimal places
export_df[columns_to_round_1] = export_df[columns_to_round_1].round(1)

# Explicit column order for Excel integration
column_order = [
    "PLAYER", "SALARY", "PROBABILITY", "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", 
    "CUT_PERCENTAGE", "OWGR_RANK", "FEDEX_CUP_POINTS", 
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Ensure all expected columns exist
for col in column_order:
    if col not in export_df.columns:
        export_df[col] = "-" if col == "COURSE_HISTORY" else np.nan

# Reorder columns to match Excel layout
export_df = export_df[column_order]

# Export to CSV
filename = f"data/current_week_export.csv"
export_df.to_csv(filename, index=False)
print(f"✅ Exported to {filename}")

✅ Exported to data/current_week_export.csv
